In [12]:
using Zarr
import Zarr: NoCompressor, BloscCompressor, ZlibCompressor, MaxLengthString

using Stars
using Ipaper

In [9]:
n = Int(5e3)
ntime = 20
A = rand(Int32, n, n, 20)

Array{Int32, 3} | (5000, 5000, 20) | 1907.35 Mb


In [78]:
obj_size(A)

Array{Int32, 3} | (5000, 5000, 20) | 1907.35 Mb


# 1. tiff

In [ ]:
# 第一次读写
fout = "f_star.tif"
ra = Raster(A)
@time st_write(ra, fout)

In [79]:
# 第二次读写
# 可能是tiled的原因导致文件偏大
ra = Raster(A)
@time st_write(ra, fout)

 38.123412 seconds (199 allocations: 3.725 GiB, 0.43% gc time)


"f_star.tif"

In [80]:
fout = "f_star.tif"
gdal_info(fout)

Driver: GTiff/GeoTIFF
Files: f_star.tif
Size is 5000, 5000
Origin = (0.000000000000000,0.000000000000000)
Pixel Size = (1.000000000000000,1.000000000000000)
Image Structure Metadata:
  COMPRESSION=DEFLATE
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  (   0.0000000,   0.0000000) 
Lower Left  (       0.000,    5000.000) 
Upper Right (    5000.000,       0.000) 
Lower Right (    5000.000,    5000.000) 
Center      (    2500.000,    2500.000) 
Band 1 Block=256x256 Type=Int32, ColorInterp=Gray
Band 2 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 3 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 4 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 5 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 6 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 7 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 8 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 9 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 10 Block=256x256 Type=Int32, ColorInterp=Undefined
Band 

In [85]:
## 采用Terra进行读写
import Terra

b = Terra.bbox(70, 15, 140, 55)
r2 = Terra.Raster(A, b);

5000×5000×20 Raster{Int32,3} with dimensions: 
  X Sampled{Float64} 70.007:0.014:139.993 ForwardOrdered Regular Points,
  Y Sampled{Float64} 54.996:-0.008:15.004 ReverseOrdered Regular Points,
  Ti Sampled{Int64} 1:20 ForwardOrdered Regular Points
extent: Extent(X = (70.007, 139.993), Y = (15.004, 54.996), Ti = (1, 20))parent:
[:, :, 1]
                   54.996  …           15.012           15.004
  70.007   1467646213          -651358032      -1268503273
  70.021   1766827902         -1111082297        506205614
  70.035  -1683672301         -1975199098      -1513001614
  70.049   2130086526           388366914       -295741113
  70.063    967114387      …   -402836693       1625088659
  70.077   -891420784         -1902265717      -1104058886
  70.091    309119059          2146959031       -990137432
   ⋮                       ⋱            ⋮      
 139.895  -1355595409          1784873726       -995661444
 139.909   1860435156      …    896685357      -2083997387
 139.923  -21420133

In [88]:
# @time Terra.st_write(r2, "f_terra.tif");

# 2. Zarr

In [105]:
## 编写一个保存数据的函数
using Zarr: AbstractStore, store_read_strategy, SequentialRead, ConcurrentRead
Zarr.concurrent_io_tasks[]

# add the `overwrite` option to zarr
## DirectoryStore
function zarr_group(s::String; overwrite=false, mode="w", kw...)
    if overwrite && isdir(s)
        rm(s, recursive=true, force=true)
    end
    isdir(s) ? zopen(s, mode) : zgroup(s; kw...)
end


p = "f_zarr-01.zarr"
g = zarr_group(p * "/LAI")

ZarrGroup at DirectoryStore("f_zarr-01.zarr/LAI") and path 
Variables: aInt32 

## 2.1 非并行版本

In [106]:
## 测试`DirectoryStore`的两种读写方案
Zarr.store_read_strategy(::DirectoryStore) = SequentialRead()
# Zarr.store_read_strategy(::DirectoryStore) = ConcurrentRead(Zarr.concurrent_io_tasks[])

# store_read_strategy(g.storage)

In [75]:
# A
t = nonmissingtype(eltype(A))
dims = size(A)
compressor = NoCompressor()
# how to automatically get the chunk size? 
chunks = (1000, 1000, 10)

z = zcreate(t, g, string("a", t), dims...;chunks, compressor)

ZArray{Int32} of size 5000 x 5000 x 20

In [89]:
@time z[:] .= A;

  8.995942 seconds (10.47 k allocations: 7.451 GiB, 5.25% gc time)


## 2.2 并行版本

In [107]:
## 测试`DirectoryStore`的两种读写方案
# Zarr.store_read_strategy(::DirectoryStore) = SequentialRead()
Zarr.store_read_strategy(::DirectoryStore) = ConcurrentRead(Zarr.concurrent_io_tasks[])
Zarr.concurrent_io_tasks[] = 12;

t = nonmissingtype(eltype(A))
dims = size(A)
compressor = NoCompressor()
chunks = (1000, 1000, 10)

varname = string("a", t, "_par")
z = zcreate(t, g, varname, dims...;chunks, compressor)
# store_read_strategy(g.storage)

ZArray{Int32} of size 5000 x 5000 x 20

In [110]:
g

ZarrGroup at DirectoryStore("f_zarr-01.zarr/LAI") and path 
Variables: aInt32 aInt32_par 

In [109]:
store_read_strategy(g.storage)

ConcurrentRead(12)

In [113]:
@time z[:] .= A;
# 速度没有明显的提高，但读写速度还挺快

  8.281621 seconds (20.70 k allocations: 7.452 GiB, 5.29% gc time)


## 2.3 压缩


In [130]:
g

ZarrGroup at DirectoryStore("f_zarr-01.zarr/LAI") and path 
Variables: aInt32 aInt32_compress aInt32_par 

In [129]:
# methods(zcreate)
# z = g["aInt32_compress"]
# path for a variable
path = z.storage.folder * "/" * z.path

rm(path, recursive=true)

In [131]:
varname = string("a", t, "_compress")
compressor = BloscCompressor(cname="zstd",shuffle=0)

# I want to overwrite this variable
z = zcreate(t, g, varname, dims...;chunks, compressor)

ZArray{Int32} of size 5000 x 5000 x 20

In [132]:
# 这种随机的数据，压缩也没用
@time z[:] .= A;

 10.907466 seconds (20.25 k allocations: 7.452 GiB, 3.16% gc time)


## 2.4 设计MODIS全球计算框架

并行处理需要在实战中设计与改进。

In [134]:
## 全球有112个tiff，需要记录那个文件是处理过的。
var_data = "data"
var_record = var_data * "_record"

"data_record"

In [148]:
# z.attrs = Dict("Info" => info)
z = zcreate(t, g, var_data, dims...;chunks, attrs=Dict("Info" => info), compressor)


ZArray{Int32} of size 5000 x 5000 x 20

In [157]:
z.attrs["Info"]["1"] = 999

999

In [158]:
z
## 采用dict的方式记录信息

ZArray{Int32} of size 5000 x 5000 x 20

In [155]:
import DataStructures: OrderedDict

# z = zcreate(t, g, varname, dims...;chunks, compressor)
# 需要一个dict记录哪些文件是保存成功的，哪些没有
x = collect(1:12)
fs = string.(x)
info = OrderedDict(zip(fs, x))

OrderedDict{String, Int64} with 12 entries:
  "1"  => 1
  "2"  => 2
  "3"  => 3
  "4"  => 4
  "5"  => 5
  "6"  => 6
  "7"  => 7
  "8"  => 8
  "9"  => 9
  "10" => 10
  "11" => 11
  "12" => 12